In [1]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pl.read_csv("raw/Data_Sources_csv.csv", separator=";")
data.head(5)

url,date,text,tag,source
str,str,str,str,str
"""https://vk.com…","""2023-04-15 19:…","""Всем добрый де…","""БАР""","""ВК"""
"""https://vk.com…","""2023-04-15 19:…","""это на всякий …","""БАР""","""ВК"""
"""https://vk.com…","""2023-04-15 19:…","""Попробуйте усп…","""БАР""","""ВК"""
"""https://vk.com…","""2023-04-15 19:…","""На все вопросы…","""БАР""","""ВК"""
"""https://vk.com…","""2023-04-15 19:…","""Всё к лучшему.…","""БАР""","""ВК"""


In [3]:
print(len(data))
data = data.drop_nulls()
print(len(data))

66460
65893


In [4]:
data["tag"].value_counts()

tag,count
str,u32
"""БАР""",2794
"""шизофрения""",2055
"""тревожное р-во…",1
"""депрессия""",37475
"""паранойя""",905
"""тревожное р-во…",2217
"""тревожное р-во…",10069
"""ОКР""",5158
"""ПРЛ""",5219


Т.к. разрешено оставить не менее 6 классов, уберем "тревожное р-во/невроз", "тревожное р-во/депрессия", "паранойя"

In [5]:
data = data.filter(~pl.col("tag").is_in(["тревожное р-во/невроз", "тревожное р-во/депрессия", "паранойя"]))
data.shape

(62770, 5)

In [6]:
data["tag"].value_counts()

tag,count
str,u32
"""ПРЛ""",5219
"""ОКР""",5158
"""тревожное р-во…",10069
"""шизофрения""",2055
"""депрессия""",37475
"""БАР""",2794


In [7]:
data["text"].to_list()[:10]

['Всем добрый день!  У меня БАР 2 типа. Начала часто употреблять спиртные напитки, при этом не помню что происходило. Именно из-за этого несколько раз ругалась с мужем (творила всякую дичь, вплоть до звонков бывшему, чего не помню), в последний раз сильно начали ругаться, не помню из-за чего, потом помню лишь чувство страха и бывший появляется у двери нашей квартиры. Муж взял несколько своих вещей и ушёл. Домой не возвращается уже седьмой день, на контакт не идет вообще. Что делать? Не могу ни спать, ни есть, дикие истерики первые 4 дня были вплоть до самоповреждения. Сейчас полегче, но истерики продолжаются (1 раз в 1-2 дня, ближе к ночи). Очень хочу всё исправить, вернуть любимого, неизвестность тревожит, понимаю, что очень накосячила, переосмыслила всю свою жизнь, отношения, нашла миллион косяков со своей стороны, готова всё исправить, начинаю работать над собой, ищу хорошего психиатра (прошлый психиатр упорно назначал антидепрессанты, с которых мне становилось хуже). Помогите пожал

### Уменьшение и балансировка датасета

In [8]:
target_total_count = 25000

# Определите количество уникальных тегов
unique_tags = data['tag'].n_unique()

# Установите целевое количество строк для каждого класса
target_count_per_class = target_total_count // unique_tags

data = data.with_columns((pl.col('text').str.split(' ').list.lengths()).alias('word_count'))

# Функция для выборки подмножества данных для каждого класса по наибольшему количеству слов
def select_top_by_word_count(df: pl.DataFrame, target_count):
    return df.sort('word_count', descending=True).head(target_count)

# # Функция для выборки подмножества данных для каждого класса
# def sample_class(df, target_count):
#     if df.shape[0] > target_count:
#         return df.sample(n=target_count)
#     return df

# Создайте новый сбалансированный датасет
balanced_dfs = []

for tag in data['tag'].unique():
    class_df = data.filter(pl.col('tag') == tag)
    sampled_class_df = select_top_by_word_count(class_df, target_count_per_class)
    balanced_dfs.append(sampled_class_df)

# Объедините все классы в один датасет
balanced_df = pl.concat(balanced_dfs)

# Если суммарное количество строк больше нужного, случайно уменьшите его до нужного размера
if balanced_df.shape[0] > target_total_count:
    balanced_df = balanced_df.sample(n=target_total_count)

balanced_df = balanced_df.sample(fraction=1, shuffle=True)

C:\Users\ciril\AppData\Local\Temp/ipykernel_18028/2204275714.py:9: DeprecationWarning: `lengths` is deprecated. It has been renamed to `len`.
  data = data.with_columns((pl.col('text').str.split(' ').list.lengths()).alias('word_count'))


In [9]:
balanced_df["tag"].value_counts()

tag,count
str,u32
"""ОКР""",4166
"""тревожное р-во…",4166
"""БАР""",2794
"""ПРЛ""",4166
"""депрессия""",4166
"""шизофрения""",2055


In [10]:
balanced_df.shape

(21513, 6)

In [11]:
balanced_df["text"].to_list()[:10]

['Добрый день, как я могу к Вам обращаться? Если согласны с условиями демо-консультации, то можем начать. Условия проведения демо-консультацииДобрый день.Предлагаю проведение демо-консультации в свободном формате в течение 3 дней. -Отвечаю в течение дня. -Если от Вас нет ответа в течение суток - закрываю тему.В результате демо-консультации Вы найдете вариант решения Вашего запроса, либо сформулируете новый запрос. Если консультация оказалась для Вас полезной, буду благодарна за отзыв.https://www.b17.ru/my_comments.php?mod=response&id=813789&prt=813789Вы должны предупредить меня, если имеете проблемы со здоровьем психического характера, зависимостях, суицидальном поведении.Демо-консультация поможет начать работу с Вашей проблемой, но не заменит личную терапию длительного характера.',
 ' Возможно, что срывы случаются в следствии приема психотропных средств, ведь прием таких средств просто "подмораживает" психику и никак не влияет на причину дисбаланса психики и в какой то момент "подморо

### Train val test split

In [12]:
tag_counts = balanced_df.groupby('tag').agg(pl.col('tag').count().alias('tag_count'))
balanced_df = balanced_df.join(tag_counts, on='tag', how='left')
balanced_df.head(3)

C:\Users\ciril\AppData\Local\Temp/ipykernel_18028/1634387339.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  tag_counts = balanced_df.groupby('tag').agg(pl.col('tag').count().alias('tag_count'))


url,date,text,tag,source,word_count,tag_count
str,str,str,str,str,u32,u32
"""https://www.b1…",""" 22.06.2022-21…","""Добрый день, к…","""тревожное р-во…","""forum b17""",92,4166
"""https://www.b1…",""" 29.04.2022-16…",""" Возможно, что…","""тревожное р-во…","""forum b17""",46,4166
"""https://www.b1…",""" 1.04.2023-23:…","""Сказал, в том …","""депрессия""","""forum b17""",108,4166


In [13]:
from sklearn.model_selection import train_test_split

X = balanced_df.select(["url", "date", "text", "source"])
y = balanced_df.select("tag")

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=balanced_df["tag_count"].to_numpy())

c:\Users\ciril\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [14]:
X_train.write_parquet("preprocessed/X_train.parquet")
y_train.write_parquet("preprocessed/y_train.parquet")
X_test.write_parquet("preprocessed/X_test.parquet")
y_test.write_parquet("preprocessed/y_test.parquet")

# TODO: катбуст на фичах:
- предикты из bert-like классификатора на текстах
- url через label encoder
- source*

### Clear texts